In [1]:
import gym 
import numpy as np
from custom_envs.gridworlds import WindyGridworldEnv, SimpleGridworldEnv
from custom_envs.mountain_car import MountainCarEnv

from function_approximators.function_approximators import NeuralNetwork, LinearModel, DecisionTree, RandomForest, SupportVectorRegressor, KNeighboursRegressor, GaussianProcess, OnlineGaussianProcess
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, RationalQuadratic, Matern
from sklearn.metrics.pairwise import rbf_kernel, chi2_kernel, laplacian_kernel 

from utils.train_utils import train, solve, train_time
from utils.plot_utils import plot_returns

from agents.av_agents import DQNAgent, LinearAgent, FQIAgent, OnlineGaussianProccessAgent

import operator


In [2]:
function_approximators = [NeuralNetwork, LinearModel, DecisionTree, RandomForest, SupportVectorRegressor, KNeighboursRegressor, GaussianProcess, OnlineGaussianProcess]

agents = [DQNAgent, LinearAgent, *[FQIAgent]*5, OnlineGaussianProccessAgent]

RENDER = False
# env = gym.make("CartPole-v1")
# env = gym.make("Acrobot-v1")
# env = gym.make("MountainCar-v0")
# env = WindyGridworldEnv()
# env = gym.make("LunarLander-v2")
env = SimpleGridworldEnv()
# env = MountainCarEnv()

In [3]:
# DQN Config
CONFIG_DQN = {
    "episode_length": 20,
    "max_timesteps": 2000,
    "max_time": 30 * 60,
    "eval_freq": 100, 
    "eval_episodes": 10,
    "learning_rate": 0.00075,
    "hidden_size": (32,32),
    "target_update_freq": 20,
    "batch_size": 32,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "plot_loss": False,
    "epsilon": 1,
    "max_deduct": 0.97,
    "decay": 0.25,
    "lr_step_size": 100,
    "lr_gamma": 0.95,
    "max_steps": 20,
    "non_param": False,
}

# Linear Config
CONFIG_LINEAR = {
    "episode_length": 20,
    "max_timesteps": 2000,
    "max_time": 30 * 60,
    "eval_freq": 100, 
    "eval_episodes": 10,
    "learning_rate": 0.02,
    "target_update_freq": 10,
    "batch_size": 32,
    "gamma": 0.99,
    "buffer_capacity": int(1e7),
    "plot_loss": False,
    "epsilon": 1,
    "max_steps": 20,
    "poly_degree": 1,
    "max_deduct": 0.97,
    "decay": 0.5,
    "lr_step_size": 100,
    "lr_gamma": 0.99,
    "non_param": False,
}

# Decision Tree Config
CONFIG_DT = {
    "episode_length": 20,
    "max_timesteps": 2000,
    "max_time": 30 * 60,
    "eval_freq": 100, 
    "eval_episodes": 10,
    "model_save_freq": 250,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.2,
    "max_steps": 20,
    "non_param": True,
    "model_params": {"criterion":"mse","max_depth": 10, "min_samples_split": 20, "min_samples_leaf": 5},
    "feature_names": ["Cart Position", "Cart Velocity", "Pole Angle", "Pole Angular Velocity", "Action: Push Left", "Action: Push Right"],
    "plot_name": "dt_depth=8",
}

# Random Forest Config
CONFIG_RF = {
    "episode_length": 20,
    "max_timesteps": 2000,
    "max_time": 30 * 60,
    "eval_freq": 100, 
    "eval_episodes": 5,
    "model_save_freq": 250,
    "model_save_capacity": 20,
    "update_freq": 5,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.2,
    "max_steps": 20,
    "non_param": True,
    "model_params": {"n_estimators": 10,"max_depth": 10, "min_samples_split": 20, "min_samples_leaf": 5},
}

# Support Vector Regressor Config
CONFIG_SVR = {
    "episode_length": 20,
    "max_timesteps": 2000,
    "max_time": 30 * 60,
    "eval_freq": 100, 
    "eval_episodes": 10,
    "model_save_freq": 250,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 256,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 20,
    "non_param": True,
    "model_params": {"kernel":"rbf", "degree": 2, "C": 2},
}


# K-Neighbors Regressor Config
CONFIG_KNR = {
    "episode_length": 20,
    "max_timesteps": 2000,
    "max_time": 30 * 60,
    "eval_freq": 100, 
    "eval_episodes": 5,
    "model_save_freq": 250,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 256,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 20,
    "non_param": True,
    "model_params": {"n_neighbors":7, "weights": "distance", "algorithm": "auto", "leaf_size": 30},
}

# Gaussian Process Config
CONFIG_GP = {
    "episode_length": 20,
    "max_timesteps": 2000,
    "max_time": 30 * 60,
    "eval_freq": 100, 
    "eval_episodes": 5,
    "model_save_freq": 250,
    "model_save_capacity": 20,
    "update_freq": 10,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 20,
    "non_param": True,
    "model_params": {"alpha": 1e-10, "normalize_y": False, "kernel":  RBF(length_scale=0.08, length_scale_bounds="fixed")},
}

# Online Gaussian Process Config
CONFIG_GP_Online = {
    "episode_length": 20,
    "max_timesteps": 2000,
    "max_time": 30 * 60,
    "eval_freq": 100, 
    "eval_episodes": 10,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "batch_size": 32,
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 20,
    "non_param": True,
    "model_params": {"sigma_0": 0.5, "init":-5, "kernel":  rbf_kernel, "epsilon_tol": 0.07, "basis_limit": 1000},
}

In [4]:
returns = []
train_returns = []
train_times = []
n_seeds=10

j=7
for i in range(n_seeds):
    print(f"\n Run: {i+1} \n")
    r, _, t, times = train(env, 
            CONFIG_GP_Online, 
            fa=function_approximators[j], 
            agent = agents[j], 
            render=RENDER,
            online=True)
    env.close()
    returns.append(r)
    train_returns.append(t)
    train_times.append(times)
    


  2%|▏         | 38/2000 [00:00<00:05, 332.70it/s]
 Run: 1 

  6%|▌         | 118/2000 [00:00<00:06, 276.15it/s]Evaluation at timestep 118 returned a mean returns of -20.0
Epsilon = 0.8448333333333333
Support Points = 119
 12%|█▏        | 238/2000 [00:01<00:16, 104.74it/s]Evaluation at timestep 218 returned a mean returns of -20.0
Epsilon = 0.6865
Support Points = 219
 16%|█▌        | 318/2000 [00:02<00:12, 130.52it/s]Evaluation at timestep 318 returned a mean returns of -20.0
Epsilon = 0.5281666666666667
Support Points = 307
 21%|██        | 418/2000 [00:03<00:17, 90.98it/s]Evaluation at timestep 418 returned a mean returns of -20.0
Epsilon = 0.36983333333333335
Support Points = 393
 26%|██▌       | 518/2000 [00:05<00:18, 82.31it/s]Evaluation at timestep 518 returned a mean returns of -20.0
Epsilon = 0.21150000000000002
Support Points = 457
 31%|███       | 618/2000 [00:06<00:14, 92.76it/s]Evaluation at timestep 618 returned a mean returns of -20.0
Epsilon = 0.053166666666666695
Suppo

KeyboardInterrupt: 

In [9]:
env.observation_space.shape

()

In [5]:
with open(f'simplegrid_eval_Gaussian Process.csv', 'ab') as eval:
    for i in range(n_seeds):
        np.savetxt(eval, [returns[i]], delimiter=',')

In [6]:
with open(f'simplegrid_train_Gaussian Process.csv', 'ab') as train:
    for i in range(n_seeds):
        np.savetxt(train, [train_returns[i]], delimiter=',')
        np.savetxt(train, [train_times[i]], delimiter=',')

In [8]:
n_eps = []
n_steps = []
not_solved = []
n_seeds=30

j=7
for i in range(n_seeds):
    print(f"\n Run: {i+1} \n")
    s, e, n = solve(env, 
            CONFIG_GP_Online, 
            fa=function_approximators[j], 
            agent = agents[j],
            target_return=195,
            op=operator.ge, 
            render=RENDER,
            online=True)
    env.close()
    n_eps.append(e)
    n_steps.append(s)
    not_solved.append(n)


 Run: 1 

Ep. timesteps: 200
Total timesteps: 3434
Total episodes: 90
Evaluation mean return: 197.4

 Run: 2 

Ep. timesteps: 200
Total timesteps: 1388
Total episodes: 53
Evaluation mean return: 195.9

 Run: 3 

Ep. timesteps: 200
Total timesteps: 360
Total episodes: 14
Evaluation mean return: 200.0

 Run: 4 

Ep. timesteps: 200
Total timesteps: 11329
Total episodes: 168
Evaluation mean return: 196.7

 Run: 5 

Ep. timesteps: 200
Total timesteps: 5993
Total episodes: 134
Evaluation mean return: 200.0

 Run: 6 

Ep. timesteps: 200
Total timesteps: 2807
Total episodes: 82
Evaluation mean return: 200.0

 Run: 7 

Ep. timesteps: 200
Total timesteps: 1367
Total episodes: 49
Evaluation mean return: 200.0

 Run: 8 

Ep. timesteps: 200
Total timesteps: 874
Total episodes: 37
Evaluation mean return: 200.0

 Run: 9 

Ep. timesteps: 200
Total timesteps: 2291
Total episodes: 78
Evaluation mean return: 200.0

 Run: 10 

Ep. timesteps: 200
Total timesteps: 740
Total episodes: 28
Evaluation mean ret

In [9]:
with open(f'cartpole_se195_Gaussian Process Online.csv', 'ab') as se:
    np.savetxt(se, [n_eps], delimiter=',')
    np.savetxt(se, [n_steps], delimiter=',')
    np.savetxt(se, [not_solved], delimiter=',')

In [12]:
mean_eps = np.mean(n_eps)
std_eps = np.std(n_eps)
print(f"Average n_eps: {mean_eps}")
print(f"Std n_eps: {std_eps}")
print(f"St.error n_eps: {std_eps/np.sqrt(n_seeds)}")

mean_steps = np.mean(n_steps)
std_steps = np.std(n_steps)
print(f"Average n_steps: {mean_steps}0")
print(f"Std n_steps: {std_steps}")
print(f"St.error n_steps: {std_steps/np.sqrt(n_seeds)}")

print(f"Not solved: {np.sum(not_solved)} runs")

Average n_eps: 77.13333333333334
Std n_eps: 47.77219088781906
St.error n_eps: 8.721968856900416
Average n_steps: 3211.03333333333330
Std n_steps: 4515.506184126967
St.error n_steps: 824.4148652001386
Not solved: 1 runs


In [14]:
j = 0
time = train_time(env, 
        CONFIG_DQN, 
        fa=function_approximators[j], 
        agent = agents[j],
        online=True)
env.close()

print(time)

20029it [06:32, 51.03it/s]200.0
392.48629212379456

